# Make an SED!

In [1]:
from SEDkit import sed, spectrum, synphot
import astropy.units as q
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

Let's create a new SED for an M-dwarf. The software assumes a field age if no `age` argument is given.

In [2]:
s = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)


Now let's add some photometry and take a look at how it's stored!

In [3]:
# Add the band name, magnitude, and uncertainty
s.add_photometry('2MASS.J', 14.391, 0.029)
s.add_photometry('2MASS.H', 13.677, 0.032)
s.add_photometry('2MASS.Ks', 13.535, 0.034)
s.add_photometry('WISE.W1', 13.55, 0.027)
s.add_photometry('WISE.W2', 13.581, 0.035)
s.add_photometry('WISE.W3', 12.884, None)
s.add_photometry('SDSS.u', 20.486, 0.073)
s.add_photometry('SDSS.g', 17.779, 0.006)
s.add_photometry('SDSS.r', 16.431, 0.004)
s.add_photometry('SDSS.i', 15.843, 0.004)
s.add_photometry('SDSS.z', 15.514, 0.005)

# Take a look
print(s.photometry)

  band          eff         app_magnitude ...      abs_flux_unc      bandpass
                 um                       ... erg / (Angstrom cm2 s)         
-------- ------------------ ------------- ... ---------------------- --------
  SDSS.u 0.3556523841771217        20.486 ...                    0.0   SDSS.u
  SDSS.g  0.470249523890033        17.779 ...                    0.0   SDSS.g
  SDSS.r 0.6175578617515662        16.431 ...                    0.0   SDSS.r
  SDSS.i 0.7489976493830276        15.843 ...                    0.0   SDSS.i
  SDSS.z 0.8946709375325572        15.514 ...                    0.0   SDSS.z
 2MASS.J   1.23930926619334        14.391 ...                    0.0  2MASS.J
 2MASS.H 1.6494946845618255        13.677 ...                    0.0  2MASS.H
2MASS.Ks 2.1638605296287605        13.535 ...                    0.0 2MASS.Ks
 WISE.W1 3.3897048276483708         13.55 ...                    0.0  WISE.W1
 WISE.W2  4.640635531659788        13.581 ...                   

Now that our SED has all the data it needs, we can call the `results` method, which constructs the SED and caluclates the fundamental parameters.

If no `radius` is given, one will be estimated from model isochrones using the calculated `Lbol` and assumed `age`.

In [4]:
print(s.results)

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)
    param           value                unc             units    
------------- ------------------ ------------------- -------------
         name    2MASS 261832886                  --            --
          age                6.0                 4.0           Gyr
     distance             572.28               16.41            pc
     parallax             1.7474              0.0501           mas
       radius 0.8786209573091851 0.06782337214316517        solRad
spectral_type               60.0                 0.5            --
   membership                 --                  --            --
         fbol           9.41e-12            8.88e-14 erg / (cm2 s)
         mbol             16.085                0.01            --
         Lbol           3.69e+32            2.14e+31       erg / s
     Lbol_sun              -1.02               0.025            --
         Mbol              7.297 

In [5]:
fig = s.plot()

But that radius seems wrong. Let's change it...

In [6]:
s.radius = 0.6*q.Rsun, 0.05*q.Rsun

Setting radius to (<Quantity 0.6 solRad>, <Quantity 0.05 solRad>)


In [7]:
print(s.results)

    param           value          unc        units    
------------- ------------------ -------- -------------
         name    2MASS 261832886       --            --
          age                6.0      4.0           Gyr
     distance             572.28    16.41            pc
     parallax             1.7474   0.0501           mas
       radius                0.6     0.05        solRad
spectral_type               60.0      0.5            --
   membership                 --       --            --
         fbol           9.41e-12 8.88e-14 erg / (cm2 s)
         mbol             16.085     0.01            --
         Lbol           3.69e+32 2.14e+31       erg / s
     Lbol_sun              -1.02    0.025            --
         Mbol              7.297    0.033            --
         logg                4.5     0.07            --
         mass 0.8896913720017506      0.0       solMass
         Teff               4150      183             K
